In [1]:
import numpy as np
import pandas as pd

pd.options.display.max_columns = 100

In [2]:
traffic_df = pd.read_csv('cdot_traffic.csv',parse_dates=['COUNTDATE'])

traffic_df.head()

,COUNTSTATIONID,DIR,COUNTDATE,HOUR0,HOUR1,HOUR2,HOUR3,HOUR4,HOUR5,HOUR6,HOUR7,HOUR8,HOUR9,HOUR10,HOUR11,HOUR12,HOUR13,HOUR14,HOUR15,HOUR16,HOUR17,HOUR18,HOUR19,HOUR20,HOUR21,HOUR22,HOUR23
0,1,EAST,2019-11-19,4,6,5,6,13,16,43,70,117,67,61,62,70,64,65,74,84,52,49,35,23,24,17,9
1,1,EAST,2019-11-20,104,6,4,3,10,16,34,81,62,119,58,70,69,84,71,57,70,70,30,192,26,17,15,5
2,1,EAST,2019-11-21,2,2,3,4,10,14,32,71,69,66,60,52,69,345,312,97,64,70,48,46,21,18,9,4
3,1,EAST,2019-11-22,5,3,9,6,6,18,35,93,49,70,129,58,78,74,88,114,102,165,79,47,42,31,22,16
4,1,EAST,2019-11-23,8,5,3,3,9,22,26,31,105,40,38,64,45,45,63,55,57,54,55,50,224,34,19,8


In [3]:
station_df = pd.read_csv('cdot_stations.csv',index_col='COUNTSTATIONID')

station_df.head()

,Location,Latitude,Longitude
COUNTSTATIONID,,,
1,"ON SH 6 E/O SH 59, HAXTUN",40.629710,-102.572141
2,"ON I-70 W/O SH 36, AIR PARK RD, AURORA",39.739875,-104.671758
3,"ON SH 470 NW/O SH 85, SANTA FE DR, LITTLETON",39.567108,-105.054139
4,"ON SH 36 SE/O SH 170, MCCASLIN BLVD, SUPERIOR",39.946916,-105.148988
7,"ON SH 14 MI E/O CR 33, AULT",40.582532,-104.747863


In [4]:
traffic_df.shape

(237624, 27)

In [5]:
237624 * 24

5702976

In [6]:
traffic_tidy_df = pd.melt(
    traffic_df,
    id_vars = ['COUNTSTATIONID','DIR','COUNTDATE'],
    var_name = 'Hour',
    value_name = 'Count'
)

traffic_tidy_df.head()

,COUNTSTATIONID,DIR,COUNTDATE,Hour,Count
0,1,EAST,2019-11-19,HOUR0,4
1,1,EAST,2019-11-20,HOUR0,104
2,1,EAST,2019-11-21,HOUR0,2
3,1,EAST,2019-11-22,HOUR0,5
4,1,EAST,2019-11-23,HOUR0,8


In [7]:
traffic_tidy_df.shape

(5702976, 5)

In [8]:
traffic_tidy_df['Year'] = traffic_tidy_df['COUNTDATE'].dt.year

In [9]:
traffic_tidy_df['Month'] = traffic_tidy_df['COUNTDATE'].dt.month

In [10]:
station_agg_traffic = traffic_tidy_df.groupby(['COUNTSTATIONID','DIR','Year','Month']).agg({'Count':'sum'})
station_agg_traffic.head()


Count
COUNTSTATIONID DIR  Year Month       
1              EAST 2019 1      26495
                         2      21824
                         3      24799
                         4      29980
                         5      31243

In [11]:
mccaslin_east_2019 = station_agg_traffic.loc[(4,'EAST',2019,3),'Count']
mccaslin_east_2020 = station_agg_traffic.loc[(4,'EAST',2020,3),'Count']

mccaslin_west_2019 = station_agg_traffic.loc[(4,'WEST',2019,3),'Count']
mccaslin_west_2020 = station_agg_traffic.loc[(4,'WEST',2020,3),'Count']

In [12]:
east_diff = (mccaslin_east_2020 - mccaslin_east_2019) / mccaslin_east_2019
"Eastbound traffic at McCaslin was {:.2%} March 2019 to March 2020".format(east_diff)


'Eastbound traffic at McCaslin was -25.75% March 2019 to March 2020'

In [13]:
west_diff = (mccaslin_west_2020 - mccaslin_west_2019) / mccaslin_west_2019
"Westbound traffic at McCaslin was {:.2%}  March 2019 to March 2020".format(west_diff)


'Westbound traffic at McCaslin was -25.96%  March 2019 to March 2020'

In [14]:
annual_traffic_unstack = station_agg_traffic['Count'].unstack('Year')

annual_traffic_unstack.head()

Year                          2019     2020     2021
COUNTSTATIONID DIR  Month                           
1              EAST 1      26495.0  33190.0  27783.0
                    2      21824.0  28178.0  27097.0
                    3      24799.0  31093.0  27474.0
                    4      29980.0  22033.0  27633.0
                    5      31243.0  25051.0  28944.0

In [15]:
annual_traffic_unstack['PctDiff_20_19'] = (annual_traffic_unstack[2020] - annual_traffic_unstack[2019])/annual_traffic_unstack[2019]

annual_traffic_unstack.head()

Year                          2019     2020     2021  PctDiff_20_19
COUNTSTATIONID DIR  Month                                          
1              EAST 1      26495.0  33190.0  27783.0       0.252689
                    2      21824.0  28178.0  27097.0       0.291147
                    3      24799.0  31093.0  27474.0       0.253801
                    4      29980.0  22033.0  27633.0      -0.265077
                    5      31243.0  25051.0  28944.0      -0.198188

In [16]:
annual_traffic_unstack['PctDiff_20_19'].idxmin()

(105, 'EAST', 4)

In [17]:
biggest_pct_drop = annual_traffic_unstack.loc[annual_traffic_unstack['PctDiff_20_19'].idxmin(),'PctDiff_20_19']

print("The biggest drop in traffic was {:.2%}".format(biggest_pct_drop))

The biggest drop in traffic was -95.40%


In [18]:
station_df.loc[105,'Location']

'ON I-70 E/O SH 82, GLENWOOD SPRINGS'

In [19]:
c0 = traffic_df['COUNTSTATIONID'] == 105
c1 = traffic_df['DIR'] == 'EAST'
c2 = traffic_df['COUNTDATE'].dt.year == 2020
c3 = traffic_df['COUNTDATE'].dt.month == 4

traffic_df.loc[c0 & c1 & c2 & c3,'COUNTDATE'].nunique()

3

In [20]:
# Use date range to enumerate all the dates from Jan 1 2019 thru Dec 31 2021
all_dates = len(pd.date_range('2019-01-01','2021-12-31'))
all_dates

1096

In [21]:
# Group by station ID and direction and aggregate the number of unique dates
station_date_counts = traffic_df.groupby(['COUNTSTATIONID','DIR']).agg({'COUNTDATE':'nunique'})

# Check if the number of unique dates is less than all the dates that should be there
lessthan_true_dates = station_date_counts['COUNTDATE'] < all_dates

# What fraction of stations have less than the number of all_dates
lessthan_true_dates.sum() / len(station_date_counts)

1.0

In [22]:
(station_date_counts/all_dates).mean()

COUNTDATE    0.934527
dtype: float64

In [ ]:
unique_lat_longs = pd.pivot_table(
    data = cdot_df,
    index = 'COUNTSTATIONID',
    columns = ['DIR'],
    values = ['Latitude','Longitude'],
    aggfunc = 'nunique'
)

unique_lat_longs.stack().sort_values('Latitude',ascending=False)